In [1]:
import sqlite3
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from langdetect import detect
# from zipfile import ZipFile
# from collections import Counter

In [2]:
# Input
db_path = '../Data/phonereviews.db'
db_name = 'phonereviews'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [3]:
df = pd.read_sql_query("SELECT * from " + str(db_name), conn)
df.head()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING
0,_:node89ce3018196554d41bceaeac30a31e,https://www.techworld.com/review/smartphones/l...,"""The smartphone market is as competitive as ev...",None,None,None,None
1,_:nodec8b9fba77ad36d9d11460c11f9151d9,https://www.technospain.es/fundas-smartphones-...,None,_:node46ac847e2080dc78cf9ae447647ac8d4,None,None,None
2,_:node46ac847e2080dc78cf9ae447647ac8d4,https://www.technospain.es/fundas-smartphones-...,None,None,"""5""@es-es","""5""@es-es",None
3,_:nodef160bce2dcba31bd02d1a072e6ad2,https://www.technospain.es/fundas-smartphones-...,None,_:nodef4bf4b94afaabab1bbb342d3ccc284af,None,None,None
4,_:nodef4bf4b94afaabab1bbb342d3ccc284af,https://www.technospain.es/fundas-smartphones-...,None,None,"""2""@es-es","""5""@es-es",None


In [12]:
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING
count,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# drop duplicates
df.dropna()
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING
count,804458,804458,308272,369753,365588,139590,92655
unique,804374,86495,68900,368902,2027,208,156
top,_:nodea85437d748f4b0c6b7496a31b0b3ebbb,https://my-phone-finder.com/find-imei,"""Cancel this order""","""Null""@ja","""5""","""5""","""0"""
freq,2,3590,31383,82,193872,33364,26965


In [63]:
# drop rows where any of the values is null
df = df[df.REVIEWBODY.notnull()&df.RATING.notnull()&df.REVIEWRATING.notnull()&df.BESTRATING.notnull()&df.WORSTRATING.notnull()]
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING
count,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# drop duplicates on Reviewbody
df = df.drop_duplicates(subset='REVIEWBODY', keep="first")
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING
count,68901,68901,68900,46555,0,0,0
unique,68900,12619,68900,46328,0,0,0
top,_:nodeb81aca48a7d95e968aa3d136cd4565,https://my-phone-finder.com/check-phone,"""I would like to thank you very much for the q...","""9 ""@en-GB",NaN,NaN,NaN
freq,2,1282,1,25,NaN,NaN,NaN


In [6]:
# detect the language of the reviewbody
def detectLang(row):
    try:
        return detect(row['REVIEWBODY'])
    except:
        print("exception:", row['REVIEWBODY'])
    else:
        print("sad:", row['REVIEWBODY'])
df['LANGUAGE'] = df.apply(detectLang, axis=1)

exception: None


In [7]:
# filter out the ones which could not be detected or which are not english
df = df[df['LANGUAGE'] == "en"]
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING,LANGUAGE
count,67633,67633,67633,45559,0,0,0,67633
unique,67632,12479,67633,45332,0,0,0,1
top,_:nodeb81aca48a7d95e968aa3d136cd4565,https://my-phone-finder.com/check-phone,"""I would like to thank you very much for the q...","""9 ""@en-GB",NaN,NaN,NaN,en
freq,2,1251,1,25,NaN,NaN,NaN,67633


In [8]:
# finding out where the reviews come from (domain specific)
def getNetloc(row):
    try:
        return urlparse(row['URL']).netloc
    except:
        print("expection: ", row['URL'])
    else:
        print("sad", row['URL'])

df['netloc'] = df.apply(getNetloc, axis = 1)
df.head()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING,LANGUAGE,netloc
0,_:node89ce3018196554d41bceaeac30a31e,https://www.techworld.com/review/smartphones/l...,"""The smartphone market is as competitive as ev...",None,None,None,None,en,www.techworld.com
9,_:node932ee4db8fed7d2b6e36f387762ccae4,https://www.cio.com/article/3101868/smartphone...,"""\n\t\n\t\n\t\n\t\n\n\t\nLenovo's Motorola Mob...",None,None,None,None,en,www.cio.com
30,_:node76a0665e193f5656e178f4591906334,https://www.coolsmartphone.com/2016/03/01/hono...,"""Recently I was at the Honor 5X European launc...","""85 ""@en-GB",None,None,None,en,www.coolsmartphone.com
32,_:node9e6b599bb0a0c282a5cad42fa35cea60,https://www.techradar.com/reviews/vernee-v2-pr...,"""\n\n\nWhere to buy?Online Chinese retailer, G...",_:nodec31a287f3d1864bfab3ae61cfb3bc,None,None,None,en,www.techradar.com
42,_:node618abc17c8fc753a4660a9dd8d791d92,https://www.proporta.com/gripstick-dashboard-m...,"""Another useful gadget which has yet to be ful...",_:node89349b98f280309385d7a2d8bbf43095,None,None,None,en,www.proporta.com


In [10]:
df['netloc'].value_counts().index.to_list()

['allphonespro.com',
 'my-phone-finder.com',
 'www.proporta.co.uk',
 'iphone7mobiledeals.co.uk',
 'www.bhphotovideo.com',
 'justbuylah.com',
 'www.cellphonerepair.com',
 'www.pixaprints.co.uk',
 'bestbuyheadphones.net',
 'phoneky.com',
 'www.viewpoints.com',
 'www.generationtechnologies.co.ke',
 'www.eversobling.com',
 'www.aiphone.it',
 'rechargedmyphone.co.uk',
 'www.audio-technica.com',
 'www.apple.com',
 'www.7dayshop.com',
 'www.proporta.com',
 'phoneky.co.uk',
 'www.carphonewarehouse.com',
 'www.cdw.com',
 'equipboard.com',
 'www.samsung.com',
 'www.sony.com',
 'fitblizz.gocircle.net',
 'www.pssl.com',
 'www.lg.com',
 'www.coolsmartphone.com',
 'themrphone.com',
 'www.iqonlinetraining.com',
 'www.digitec.ch',
 'www.worldwidestereo.com',
 'www.macworld.com',
 'zadbuy.com',
 'www.complaintsboard.com',
 'www.techadvisor.co.uk',
 'wistex.solutions',
 'bag2me.com',
 'allaboutwindowsphone.com',
 'www.macworld.co.uk',
 'www.bestbuy.ca',
 'www.ligo.co.uk',
 'www.juno.co.uk',
 'aliexpress

In [40]:
# Clean up if the values are not accepable, e.g Null@en
df = df[df.REVIEWRATING!='"Null"@en']
df = df[df.REVIEWRATING!='"Null"']
df = df[df.BESTRATING!='"Null"@en-US']
df = df[df.BESTRATING!='"Null"']
df = df[df.WORSTRATING!='"Null"@en-US']
df = df[df.WORSTRATING!='"Null"']
df.head()
df.describe()

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING,LANGUAGE,netloc
count,35137,35137,35137,35137,35137,35137,35137,35137,35137
unique,35137,13647,35137,20595,72,8,6,1,9
top,_:znodeGTbPrfEow9J7kbPSiOG49NY9Mxo47tU,https://www.imdb.com/search/title/?title_type=...,""" Based on 2011's movie, Starbuck, this is Vau...","{'@type': 'Rating', 'worstRating': '1', 'bestR...","""3.0""@en",10,1,en,www.imdb.com
freq,1,953,1,2724,2802,14552,14552,35137,14552


In [45]:
#df['REVIEWRATING'].value_counts().index.tolist()

df['BESTRATING'].value_counts().index.tolist()

#df['WORSTRATING'].value_counts().index.tolist()

['10',
 '"4.0"@en',
 '"10"@en-US',
 '"99"@en-US',
 '"5"@en',
 '"4.0"@en-US',
 '"5"@en-US',
 '"4"@en-CA']

In [50]:
def cutout(value):
    try:
        value = value.split('&')[0]
    except:
        print("Error:" + value)

df['REVIEWRATING'].apply(cutout)

106197    None
106199    None
106201    None
106203    None
106205    None
          ... 
213531    None
213532    None
213533    None
213534    None
213535    None
Name: REVIEWRATING, Length: 35137, dtype: object

In [ ]:
phone_lst = ['smartphone', 'phone',
             'phone case', 'phone cable', 'phone charger', 'phone mount',
             'cell phone', 'mobile phone']
contains = [df_en['a'].str.contains(i) for i in phone_lst]
result = df_en[np.all(contains, axis=0)]

# df_phone = df_en[df_en['REVIEWBODY'].isin(phone_lst)]
# print(df_phone.head())
# ANGUAGE'] == "en"]
df_en.head()